In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten,Dense,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2

In [ ]:
!nvidia-smi

In [ ]:
train = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
train.head()

In [ ]:
train.info()

In [ ]:
sns.countplot(train['label'])

In [ ]:
train['label'] = train['label'].astype(str) #we are converting the label column into string as we will be using this to feed into the ImageDataGenerator

In [ ]:
train.info()

In [ ]:
json = pd.read_json('../input/cassava-leaf-disease-classification/label_num_to_disease_map.json',typ = 'series')

In [ ]:
json

In [ ]:
IMG_SIZE = 256

In [ ]:
train_datagen = image.ImageDataGenerator(rotation_range = 180,
                                  width_shift_range = 0.1,
                                  height_shift_range = 0.1,
                                  brightness_range = [0.1,1.1],
                                  horizontal_flip = True,
                                  vertical_flip = True,
                                  rescale = 1./255,
                                  zoom_range = 0.5,
                                  validation_split = 0.2)
val_datagen = image.ImageDataGenerator(rescale=1./255,
                                      validation_split = 0.2)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(dataframe = train,
                                                   directory = "/kaggle/input/cassava-leaf-disease-classification/train_images",
                                                   x_col = 'image_id',
                                                   y_col = 'label',
                                                   target_size = (IMG_SIZE,IMG_SIZE),
                                                   color_mode = "rgb",
                                                   class_mode = "categorical",
                                                   batch_size = 64,
                                                   shuffle = True,
                                                   subset = 'training')

val_generator = val_datagen.flow_from_dataframe(dataframe = train,
                                                   directory = "/kaggle/input/cassava-leaf-disease-classification/train_images",
                                                   x_col = 'image_id',
                                                   y_col = 'label',
                                                   target_size = (IMG_SIZE,IMG_SIZE),
                                                   color_mode = "rgb",
                                                   class_mode = "categorical",
                                                   batch_size = 64,
                                                   shuffle = True,
                                                   subset = 'validation')

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu', padding = 'Same',input_shape = [IMG_SIZE,IMG_SIZE,3]),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(128, (3,3), activation = 'relu',padding = 'Same'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(128, (3,3), activation = 'relu',padding = 'Same'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(128, (3,3), activation = 'relu',padding = 'Same'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(256, (3,3), activation = 'relu',padding = 'Same'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024,activation = 'relu'),
    tf.keras.layers.Dense(5, activation = 'softmax')
])

In [ ]:
from tensorflow.keras.optimizers import RMSprop,Adam
optimizer = Adam(lr = 0.001)
model.compile(loss = 'categorical_crossentropy',
              optimizer = optimizer,
             metrics = ['accuracy'])

In [ ]:
epochs = 10
batch_size = 64

In [ ]:
model.summary()

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', min_delta = 0.001, 
                           patience = 5, mode = 'min', verbose = 1,
                           restore_best_weights = True)

In [ ]:
history = model.fit_generator(train_generator,epochs = epochs,validation_data = val_generator,callbacks = early_stop)

In [ ]:
fig , ax = plt.subplots(1,2)
train_acc = history.history['accuracy']
train_loss = history.history['loss']
fig.set_size_inches(12,4)

ax[0].plot(history.history['accuracy'])
ax[0].plot(history.history['val_accuracy'])
ax[0].set_title('Training Accuracy vs Validation Accuracy')
ax[0].set_ylabel('Accuracy')
ax[0].set_xlabel('Epoch')
ax[0].legend(['Train', 'Validation'], loc='upper left')

ax[1].plot(history.history['loss'])
ax[1].plot(history.history['val_loss'])
ax[1].set_title('Training Loss vs Validation Loss')
ax[1].set_ylabel('Loss')
ax[1].set_xlabel('Epoch')
ax[1].legend(['Train', 'Validation'], loc='upper left')

plt.show()

In [ ]:
model.save('Model1.h5')

In [ ]:
preds = []
ss = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')

for image in ss.image_id:
    img = tf.keras.preprocessing.image.load_img('../input/cassava-leaf-disease-classification/test_images/' + image)
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = tf.keras.preprocessing.image.smart_resize(img, (IMG_SIZE, IMG_SIZE))
    img = tf.reshape(img, (-1, IMG_SIZE, IMG_SIZE, 3))
    prediction = model.predict(img/255)
    preds.append(np.argmax(prediction))

submission = pd.DataFrame({'image_id': ss.image_id, 'label': preds})
submission

In [ ]:
submission.to_csv('submission.csv', index = False)